# 🚀 Fine-Tuning LLM con Unsloth + LoRA

**Modelo**: unsloth/Llama-3.2-3B-Instruct (4-bit quantized)
**Dataset**: sentiment_500.json (upload como Kaggle Dataset)
**Método**: QLoRA (4-bit) con Unsloth para 2x velocidad

### Instrucciones:
1. Sube `sentiment_500.json` como un Kaggle Dataset
2. Añade ese dataset a este notebook (botón '+Add Input')
3. Selecciona GPU T4 x2 como acelerador
4. Activa Internet
5. Ejecuta todas las celdas

## Cell 1: Setup & Instalación

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes triton
!pip install minio
print('✅ Instalación completada')

## Cell 2: Imports & Config

In [ ]:
import torch
import json
import os
import time
from datetime import datetime
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# ═══════════════════════════════════════════
# 📝 CONFIGURACIÓN - Edita estos valores
# ═══════════════════════════════════════════
MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"  # Modelo preoptimizado 4-bit
MAX_SEQ_LENGTH = 2048
EPOCHS = 3
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
LORA_R = 16
LORA_ALPHA = 16

# Ruta al dataset (ajusta según cómo lo subas)
# Si lo subes como Kaggle Dataset, la ruta será algo como:
# /kaggle/input/{tu-usuario}-{nombre-dataset}/{archivo}.json
DATASET_PATH = "/kaggle/input/"  # <-- AJUSTA ESTA RUTA

# Buscar el JSON automáticamente
for root, dirs, files in os.walk("/kaggle/input/"):
    for f in files:
        if f.endswith(".json"):
            DATASET_PATH = os.path.join(root, f)
            break

print(f"📂 Dataset encontrado: {DATASET_PATH}")
print(f"🧠 Modelo: {MODEL_NAME}")
print(f"🔧 LoRA r={LORA_R}, alpha={LORA_ALPHA}")
print(f"📊 Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, LR: {LEARNING_RATE}")
print(f"🖥️ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB" if torch.cuda.is_available() else '')

## Cell 3: Cargar Modelo (4-bit Quantized)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect: float16 para T4, bfloat16 para Ampere+
    load_in_4bit=True,
)
print(f"✅ Modelo cargado: {MODEL_NAME}")

## Cell 4: Aplicar LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Contar parámetros entrenables
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA aplicado: {trainable:,} params entrenables / {total:,} total ({100*trainable/total:.2f}%)")

## Cell 5: Preparar Dataset

In [ ]:
# Cargar JSON
with open(DATASET_PATH, 'r') as f:
    raw_data = json.load(f)

print(f"📊 Dataset cargado: {len(raw_data)} ejemplos")

# Mostrar distribución de clases
from collections import Counter
label_dist = Counter(item['label'] for item in raw_data)
print(f"📊 Distribución: {dict(label_dist)}")

# Formatear como instrucciones para el modelo
PROMPT_TEMPLATE = """### Instruction:
Classify the sentiment of the following text.

### Input:
{text}

### Response:
{label}"""

formatted_data = []
for item in raw_data:
    formatted_data.append({
        "text": PROMPT_TEMPLATE.format(text=item["text"], label=item["label"])
    })

dataset = Dataset.from_list(formatted_data)
print(f"✅ Dataset formateado: {len(dataset)} ejemplos")
print(f"\n📝 Ejemplo:")
print(dataset[0]['text'])

## Cell 6: Configurar Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION,
        warmup_steps=5,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        report_to="none",
    ),
)
print("✅ Trainer configurado")

## Cell 7: 🔥 Entrenar

In [ ]:
start_time = time.time()
print(f"🏋️ Entrenamiento iniciado: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 50)

trainer_stats = trainer.train()

elapsed = time.time() - start_time
print("=" * 50)
print(f"✅ Entrenamiento completado en {elapsed/60:.1f} minutos")
print(f"📉 Training Loss Final: {trainer_stats.training_loss:.4f}")

## Cell 8: Evaluación Rápida

In [ ]:
# Probar el modelo fine-tuned
FastLanguageModel.for_inference(model)

test_texts = [
    "I absolutely love this product! Best purchase ever.",
    "Terrible experience. Complete waste of money.",
    "It was okay, nothing special but does the job.",
    "Amazing quality and fast delivery, highly recommended!",
    "Broken on arrival, very disappointed.",
]

print("🧪 Evaluación con textos de prueba:")
print("=" * 60)

for text in test_texts:
    prompt = f"""### Instruction:
Classify the sentiment of the following text.

### Input:
{text}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.1)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extraer solo la respuesta
    response = result.split("### Response:")[-1].strip().split("\n")[0]
    print(f"  Input: {text[:50]}...")
    print(f"  Pred:  {response}")
    print()

## Cell 9: Exportar LoRA Adapters (~20MB)

In [ ]:
LORA_OUTPUT = "/kaggle/working/lora_model"
model.save_pretrained(LORA_OUTPUT)
tokenizer.save_pretrained(LORA_OUTPUT)

# Calcular tamaño
lora_size = sum(os.path.getsize(os.path.join(LORA_OUTPUT, f)) for f in os.listdir(LORA_OUTPUT))
print(f"✅ LoRA adapters guardados en: {LORA_OUTPUT}")
print(f"💾 Tamaño: {lora_size / 1e6:.1f} MB")
print(f"📁 Archivos: {os.listdir(LORA_OUTPUT)}")

## Cell 10: Exportar GGUF (para uso local con llama.cpp/Ollama)

In [ ]:
# Exportar a GGUF q4_k_m (buen balance calidad/tamaño)
GGUF_OUTPUT = "/kaggle/working/gguf_model"

try:
    model.save_pretrained_gguf(
        GGUF_OUTPUT,
        tokenizer,
        quantization_method="q4_k_m",
    )
    
    # Mostrar archivos generados
    for f in os.listdir(GGUF_OUTPUT):
        size = os.path.getsize(os.path.join(GGUF_OUTPUT, f)) / 1e9
        print(f"  {f}: {size:.2f} GB")
    print(f"✅ GGUF exportado a: {GGUF_OUTPUT}")
except Exception as e:
    print(f"⚠️ GGUF export falló (puede requerir más RAM): {e}")
    print("Los LoRA adapters siguen disponibles en /kaggle/working/lora_model")

## Cell 11: Guardar Métricas & Resumen

In [ ]:
# Guardar métricas del entrenamiento
metrics = {
    "model": MODEL_NAME,
    "dataset_path": DATASET_PATH,
    "dataset_size": len(raw_data),
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "training_loss": trainer_stats.training_loss,
    "training_time_minutes": elapsed / 60,
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
    "timestamp": datetime.now().isoformat(),
    "log_history": trainer_stats.metrics,
}

with open("/kaggle/working/training_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2, default=str)

print("📊 Resumen del Entrenamiento")
print("=" * 40)
print(f"  Modelo:          {MODEL_NAME}")
print(f"  Dataset:         {len(raw_data)} ejemplos")
print(f"  Epochs:          {EPOCHS}")
print(f"  Loss Final:      {trainer_stats.training_loss:.4f}")
print(f"  Tiempo:          {elapsed/60:.1f} min")
print(f"  GPU:             {torch.cuda.get_device_name(0)}")
print("=" * 40)
print("\n📁 Archivos de salida en /kaggle/working/:")
print("  - lora_model/          (LoRA adapters)")
print("  - gguf_model/          (GGUF para Ollama)")
print("  - training_metrics.json")
print("\n🎉 ¡Fine-tuning completado! Descarga los archivos desde la pestaña Output.")